#### Add the perturb and question keys to my json, similar to the TOFU foramt (one paraphrase QA, and 5 pertrub Ans)

In [ ]:
import pandas as pd
import json

file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/qa_pairs_full.json'

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
# Convert to DataFrame
qa_df = pd.DataFrame(data)

In [19]:
qa_df = pd.DataFrame(data)

In [28]:
import pandas as pd
import json

file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/forget10.json'
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
# Convert to DataFrame
forget_df = pd.DataFrame(data)


file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/PII/retain90.json'
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
# Convert to DataFrame
retain_df = pd.DataFrame(data)


file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/test/test_retain_pii.json'
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
# Convert to DataFrame
test_retain_df = pd.DataFrame(data)

In [29]:
import random

qa_retain = qa_df.merge(retain_df, on='question', how='inner')

# Assuming each x in 'paraphrased_qa_pairs' is a list of 5 dicts with keys 'paraphrased_question' and 'paraphrased_answer'
retain_df['paraphrased_question'] = qa_retain['paraphrased_qa_pairs'].apply(
    lambda x: random.choice(x)['paraphrased_question']
)

retain_df['paraphrased_answer'] = qa_retain['paraphrased_qa_pairs'].apply(
    lambda x: random.choice(x)['paraphrased_answer']
)

retain_df['perturbed_answer'] = qa_retain['perturbed_qa_pairs'].apply(
    lambda x: [y['paraphrased_answer'] for y in x]
)

# forget_df['subject'] = qa_retain['full_name']
# forget_df = forget_df[['question','answer','paraphrased_question','paraphrased_answer','perturbed_answer','subject']]

In [42]:
test_retain_df.rename(columns={'pertrubed_answer':'perturbed_answer'},inplace=True)

In [44]:
import pandas as pd
import json

json_list = test_retain_df.to_dict(orient='records')
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/data/test/test_retain_pii.json'
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(json_list, f, ensure_ascii=False, indent=4)

print(f"JSON file created with {len(json_list)} objects")

JSON file created with 250 objects


Add the chosen paraphrase_question back to the qa_full dataset, such that the paraphrase eval_task works as well

In [15]:
import pandas as pd
import json

file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/qa_pairs_full2.json'

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
# Convert to DataFrame
qa_df = pd.DataFrame(data)

In [17]:
qa_df.drop(columns=['verification_issues'],inplace=True)

In [30]:
valid_cols = ['question','answer','paraphrased_question','paraphrased_answer','perturbed_answer']
forget_df = forget_df[valid_cols]
retain_df = retain_df[valid_cols]
test_retain_df = test_retain_df[valid_cols]



all_df = pd.concat([forget_df,retain_df,test_retain_df],ignore_index=True)


In [31]:
qa_df = qa_df.merge(all_df, on='question', how='inner')

In [33]:
qa_df.drop(columns=['answer_y','perturbed_answer'],inplace=True)

In [35]:
import pandas as pd
import json

json_list = qa_df.to_dict(orient='records')
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/qa_pairs_full2.json'
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(json_list, f, ensure_ascii=False, indent=4)

print(f"JSON file created with {len(json_list)} objects")

JSON file created with 2250 objects
